In [14]:
import pandas as pd

# 讀取CSV檔案
csv_file_path = 'facebook/20230404_20240403.csv' #facebook平台
file=open(csv_file_path,"rb")  
# 直接使用文件路径读取CSV文件
df = pd.read_csv(csv_file_path,encoding='latin-1',dtype=str)


print("預計爬取清單\n",df['URL'])


預計爬取清單
 0      https://www.facebook.com/160630887998827/posts...
1      https://www.facebook.com/114517875225866/posts...
2      https://www.facebook.com/88683275177/posts/101...
3      https://www.facebook.com/100044425449818/posts...
4      https://www.facebook.com/60182072999/posts/101...
                             ...                        
619    https://www.facebook.com/632576423454215/posts...
620    https://www.facebook.com/groups/10751114314476...
621    https://www.facebook.com/groups/11193809947634...
622    https://www.facebook.com/groups/30378797308649...
623    https://www.facebook.com/groups/60499648623717...
Name: URL, Length: 624, dtype: object


In [15]:
import os

# 設定你想要檢查和創建的資料夾路徑
folder_path = csv_file_path.replace('.csv','/')

# 檢查資料夾是否存在
if not os.path.exists(folder_path):
    # 如果資料夾不存在，則創建它
    os.makedirs(folder_path)
    print(f"資料夾 '{folder_path}' 已創建。")
else:
    # 如果資料夾已存在，則輸出提示信息
    print(f"資料夾 '{folder_path}' 已存在。")

資料夾 'facebook/2024-05-14-16-22-19-CST-search-csv-export/' 已創建。


In [10]:
##存一張圖片存該圖片評論檔成一個csv 以圖片id命名

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time

comtext = []
error = []
imgid = 0

errordf = {
    'id': [],
    'error': []
}

for i in df['URL']:
    
    try:
        driver = webdriver.Chrome()
        # 請替換成你要訪問的網頁的URL
        driver.get(i)
        #等待網站載入 等6秒
        driver.implicitly_wait(6)
        #點叉叉關掉登入框
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]').click()

        ##分離id
        imgid = i.split('posts/')[1]
        print('圖片id',imgid)
        
        ##分離id
        
        
        ##取圖片
        t = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img')
        t1 = t.get_attribute("outerHTML")
        #print(t1)

        # 使用 BeautifulSoup 解析 HTML 并获取 src 属性
        soup = BeautifulSoup(t1, 'html.parser')
        t2 = soup.img['src']  # 提取图片的 URL
        print(t2)

        pic = requests.get(t2)
        print(pic)
        img2 = pic.content #圖片裡的內容
        imgpath = csv_file_path.replace('.csv','/')
        imgpath = imgpath + str(imgid) + '.jpg'
        print(imgpath)
        pic_out = open(imgpath,'wb') 
        pic_out.write(img2) #將get圖片存入img2
        pic_out.close() #關閉檔案(很重要)
        ##取圖片
        
        
        ##取評論
        child_elements_comment = 0
        child_elements_name = 0

        #name_com = []

        id_comment = {
            'Name': [],
            'Comment': []
            }

        # 使用CSS选择器定位父元素，返回一个元素列表
        parent_elements = driver.find_elements(By.CSS_SELECTOR, '.x1y1aw1k.xn6708d.xwib8y2.x1ye3gou')

        # 遍历所有父元素
        for parent in parent_elements:
            # 在每个父元素下查找评论子元素
            child_elements_comment = parent.find_elements(By.CLASS_NAME, 'x1lliihq.xjkvuk6.x1iorvi4')
            # 在每个父元素下查找名字子元素
            child_elements_name = parent.find_elements(By.CLASS_NAME, 'xt0psk2')

            # 从找到的子元素中提取文本，这里假设每个父元素只有一个名字和一个评论
            for comment, name in zip(child_elements_comment, child_elements_name):
                
                id_comment['Name'].append(name.text)
                id_comment['Comment'].append(comment.text)

                print("Name:", name.text)
                print("Comment:", comment.text)
                print('-------')
                #name_com.append(pair) #原存評論

        #個別存評論    
        # 将数据转换成DataFrame
        dfok = pd.DataFrame(id_comment)
        print(dfok)

        # 将DataFrame保存为CSV文件
        idimgcsv = folder_path + imgid + '.csv' 
        dfok.to_csv(idimgcsv, index=False,encoding='utf-8')
        #個別存評論 
        
        driver.close()




        
        
        
    except Exception as e:
        errordf['id'].append(imgid)
        errordf['error'].append(f"发生错误：{e}")

        #error.append(imgid) 
        print('失敗',imgid,i)
        print(f"发生错误：{e}")
    time.sleep(6)

from pandas.core.frame import DataFrame
# 将数据转换成DataFrame
dfer = pd.DataFrame(errordf)
print(len(errordf),dfer)

#存起來 失敗
nameerror = csv_file_path.replace('.csv','/') + 'error.csv'
# 将DataFrame保存为CSV文件
dfer.to_csv(nameerror, index=False,encoding='utf-8')


圖片id 802647063797203
https://scontent-hkg4-1.xx.fbcdn.net/v/t1.6435-9/145382157_802647053797204_395952532155068011_n.png?stp=dst-png_p526x296&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_ohc=m4pQNx8rpx4Q7kNvgF6pwpl&_nc_ht=scontent-hkg4-1.xx&oh=00_AYBOk98QKJSQDrNebsGaKValrVe8uhdQo2BkRJX9PnBxIA&oe=666D4005
<Response [200]>
facebook/2024-05-14-16-22-19-CST-search-csv-export/802647063797203.jpg
Name: Sandra Tamez
Comment: Proverbs 31: 8-9
Speak up for those who cannot speak and defend the rights of the homeless. Raise your voice and do them justice; defends the rights of the poor and the needy.
-------
Name: Debbie Metcalf
Comment: This is very true
You had a chance of life your mom didn’t get a abortion so why are you not doing the same thing every child has a right to live every baby has a right too live and when you hear a heart beat that baby inside you is alive
-------
Name: Jana Colombi
Comment: It's pure evil and these are the same people who don't believe in the death penalty for horribl

圖片id 5722648571079407
失敗 5722648571079407 https://www.facebook.com/114517875225866/posts/5722648571079407
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+4238317

圖片id 294327052058141
失敗 294327052058141 https://www.facebook.com/100044425449818/posts/294327052058141
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+4238317]
	

圖片id 10158261445992408
https://scontent-tpe1-1.xx.fbcdn.net/v/t1.6435-9/141584914_10158261446002408_1145856839098323219_n.jpg?stp=dst-jpg_s600x600&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&_nc_ohc=OQ0v0ND4Lh4Q7kNvgEUolyG&_nc_ht=scontent-tpe1-1.xx&oh=00_AYDkjBRxR1LxHbMGIgG8S760wfc8nD03dohZqNqf9xrwrQ&oe=666D560A
<Response [200]>
facebook/2024-05-14-16-22-19-CST-search-csv-export/10158261445992408.jpg
Name: James JP Peters
Comment: I'm adopted. After 62 years, I was connected to my biological family. It's been a Blessing. I pray more people think of adoption as an alternative. Also, shouldn't Planned Parenthood be doing just that? Prevent abortions by teaching how condoms and the pill can be used?
-------
Name: 查看更多
Comment: Abortion is NOT Healthcare, it is murder, killing innocent unborn child can not speak or defend themselves and it should be treated as a felony under the men's law, then all people participating in these atrocities will face the GOD'S FINAL JUDGMENT c…… 查看更多
-------
Name: Ma

圖片id 10159457473453000
https://scontent-tpe1-1.xx.fbcdn.net/v/t1.6435-9/160802524_10159457470538000_8367919874868950025_n.jpg?stp=dst-jpg_s640x640&_nc_cat=101&ccb=1-7&_nc_sid=5f2048&_nc_ohc=Xt7iT_DUiKMQ7kNvgE5CRad&_nc_ht=scontent-tpe1-1.xx&oh=00_AYAj-dahB78WPzEY5T6qA4imorCCpiK2Hf8noN0WZTfzJw&oe=666D4FD8
<Response [200]>
facebook/2024-05-14-16-22-19-CST-search-csv-export/10159457473453000.jpg
Name: 訂閱者
Comment: And people will say the death penalty is cruel, but abortion is fine. It is a topsy-turvy world
-------
Name: Marie Lane
Comment: I'm in another country, I cannot believe the laws they have just passed in the US. I would donate if I could, but I haven't had an income for a year now due to covid. So sorry, but you do have my support.
-------
Name: Margaret Kincade
Comment: This is horrible, it is the most inhumane law in our nation . Praying it will be outlawed.
-------
Name: Cathy Jenkins
Comment: But they stopped that in many states because they said it wasn't humane to the pris

...

圖片id 4101830999850012
失敗 4101830999850012 https://www.facebook.com/152321828134302/posts/4101830999850012
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+4238317

圖片id 2923955434542417
失敗 2923955434542417 https://www.facebook.com/1795972530674052/posts/2923955434542417
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+423831

圖片id 1842324695921026
失敗 1842324695921026 https://www.facebook.com/219631734857005/posts/1842324695921026
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+4238317

失敗 3717627874946890 https://www.facebook.com/groups/322930696485/permalink/10157417258841486
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/2066599573384815/permalink/3871332996244788
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/1514785802083042/permalink/2964665837095024
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/146547666041036/permalink/683451839017280
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/517886748411617/permalink/1805341199666159
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/389832211059276/permalink/5275480745827707
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/389832211059276/permalink/5266192750089840
发生错误：list index out of range
失敗 3717627874946890 https://www.facebook.com/groups/289018045281940/permalink/893572318159840
发生错误：list index out of range
失敗 3717627

圖片id 1092535411189879
失敗 1092535411189879 https://www.facebook.com/280992089010886/posts/1092535411189879
发生错误：Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66F707032+63090]
	(No symbol) [0x00007FF66F672C82]
	(No symbol) [0x00007FF66F50EC65]
	(No symbol) [0x00007FF66F55499D]
	(No symbol) [0x00007FF66F554ADC]
	(No symbol) [0x00007FF66F595B37]
	(No symbol) [0x00007FF66F57701F]
	(No symbol) [0x00007FF66F593412]
	(No symbol) [0x00007FF66F576D83]
	(No symbol) [0x00007FF66F5483A8]
	(No symbol) [0x00007FF66F549441]
	GetHandleVerifier [0x00007FF66FB025AD+4238317